In [7]:
# OLD book memory
# # GT book titles with images (for our reference)
# img_titles = [
#     ["Artifical Intelligence A Modern Approach", "Russell Norvig", "Prentice Hall"], # 16
#     ["Computer Networks A Systems Approach", "Edition 3", "Peterson Davie", "Morgan Kaufmann"], # 17
#     ["Before the coffee gets cold", "Toshikazu Kawaguchi"], # 18
#     ["Crying in H Mart", "Michelle Zauner"], # 19
#     ["Wild", "Cheryl Strayed"], # 20
#     ["Readings in Artificial Intelligence and Software Engineering", "Rich Waters"], # 21
#     ["Computer Architecture A Quantitative Approach", "Hennessy Patterson"], # 22
#     ["When breath becomes air", "Paul Kalanthi"] # 23
# ]

# # image info to add
# img_info = []
# for img in range(8):
#     bgr = cv2.imread(f"spines/{img:02d}.png")
#     depth = np.zeros_like(bgr[:, :, 0], dtype=np.float32)
#     img_info.append({
#         "image": [bgr, depth]
#     })

# # book info to add
# # don't store name (should be looked up from database)
# book_info = [
#     {"id": [16, 24, 25], # AI a modern approach: 3 matches
#      "similarity": [0.2, 0.1, 0.05],
#      "confidence": [0.5, 0.5, 0.3]},
#     {"id": [17, 26],
#      "similarity": [0.7, 0.2],
#      "confidence": [0.6, 0.4]},
#     {"id": [18, 27],
#      "similarity": [0.3, 0.1],
#      "confidence": [0.4, 0.1]},
#     {"id": [19, 28],
#      "similarity": [0.4, 0.2],
#      "confidence": [0.4, 0.2]},
#     {"id": [20, 29],
#      "similarity": [0.6, 0.2],
#      "confidence": [0.7, 0.1]},
#     {"id": [21], # Readings in AI and SWE: one (correct match)
#      "similarity": [0.2],
#      "confidence": [0.2]},
#     {"id": [22, 30],
#      "similarity": [0.3, 0.3],
#      "confidence": [0.6, 0.3]},
#     {"id": [], # When breath becomes air: no matches found
#      "similarity": [],
#      "confidence": []},
# ]

# # build book memory
# bm = BookMemory()

# # add books
# for i in range(8):
#     bm.add_book(np.array([0, 0, 0]), book_info[i], img_info[i])

In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [2]:
# setup: import and book_memory
import cv2, numpy as np
import book_memory  
from book_memory import BookMemory
from database import book_database
import gradio as gr
import pickle, types, sys
from collections import defaultdict
from PIL import Image
from pathlib import Path
from PIL import ImageDraw



# NEW book memory
# --- stub top-level book_utils ---------------------------------
stub = types.ModuleType("book_utils")
stub.book_database  = book_database
stub.WagnerFischer = lambda *a, **kw: None

# --- expose book_memory under the dotted name ------------------
stub.book_memory = book_memory                   # attribute access
sys.modules["book_utils"] = stub                 # top-level
sys.modules["book_utils.book_memory"] = book_memory  # dotted path

# --- unpickle ---------------------------------------------------
with open("fails.pkl", "rb") as fh:
    bm = pickle.load(fh)

print("Loaded OK:", type(bm))

book_database["7"]


/Users/isabelsieh/Desktop/library/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded OK: <class 'book_memory.BookMemory'>


{'alt_title': '（萬曆）粤大記:32卷', 'call_number': 'DS793.K7 K88 1990', 'lang': 'CHN'}

In [3]:
bm.book_positions[1]

[array([316, 575]), array([412, 441])]

In [4]:
img_groups = {}

for idx, info in enumerate(bm.book_infos):
    img_key = id(bm.book_img_info[idx]["image"])
    # initialize: “skipped” starts as a dict mapping each between_indices value → list of idx
    if img_key not in img_groups:
        img_groups[img_key] = {"skipped": {}, "unsure": []}

    if "between_indices" in info:
        bi = info["between_indices"]
        # ensure bi is hashable; if it’s a list, convert to tuple
        if isinstance(bi, list):
            bi = tuple(bi)
        img_groups[img_key]["skipped"].setdefault(bi, []).append(idx)
    else:
        img_groups[img_key]["unsure"].append(idx)

# After collecting everything, convert each “skipped” dict → list of lists
for img_key, buckets in img_groups.items():
    skipped_dict = buckets["skipped"]
    img_groups[img_key]["skipped"] = list(skipped_dict.values())

img_keys = list(img_groups.keys())


In [6]:
img_groups

{11212771888: {'skipped': [[0], [1], [2]], 'unsure': [3]}}

In [7]:
bm.book_positions[1]

[array([316, 575]), array([412, 441])]

In [5]:

MANUAL = "Manually label book"
new_bm = BookMemory()            # “empty” BookMemory for storing final labels
call_to_id = {                    # mapping call_number → its integer ID in book_database
    rec["call_number"]: int(k)
    for k, rec in book_database.items()
}

init_state = [0, "u", 0, 0, 0]


def load_image_rgb(idx: int) -> Image.Image:
    bgr = bm.book_img_info[idx]["image"]
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(rgb)

def build_manual_choices():
    """
    Return a list of (label, value) tuples for every book in book_database,
    where label = "CALL_NUMBER, ALT_TITLE" and value = the database ID.
    """
    opts = []
    for db_id, rec in book_database.items():
        callnum   = rec.get("call_number", "")
        alt_title = rec.get("alt_title", "")
        if callnum or alt_title:
            label = f"{callnum}, {alt_title}"
            opts.append((label, db_id))
    return opts

def annotate_on_image(img, point, radius=30, color=(0, 255, 0, 64), width=2):
    img = img.convert("RGBA")

    # 2) Make a transparent overlay of the same size
    overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)

    x, y = point
    r = radius

    # 3) Draw a filled ellipse onto the overlay
    draw.ellipse(
        (x - r, y - r, x + r, y + r),
        fill=color,        # semi-transparent fill
        outline=None       # no border (optional)
    )

    # 4) Composite overlay onto the base image
    return Image.alpha_composite(img, overlay)

def annotate_skip_box(
    img: Image.Image,
    idx: int,
    outline_color=(255, 0, 0),
    outline_width=10,
):
    base = img.convert("RGBA")
    orig_w, orig_h = base.size

    # 2) Extract left/right x from book_positions, cast to int
    left_x  = int(bm.book_positions[idx][0][0])
    right_x = int(bm.book_positions[idx][1][0])

    # 4) Create transparent overlay
    overlay = Image.new("RGBA", base.size, (0, 0, 0, 0))
    draw    = ImageDraw.Draw(overlay)

    # 5) Draw only the rectangle outline
    draw.rectangle(
        [(left_x, 0), (right_x, orig_h)],
        outline=outline_color + (255,),
        width=outline_width
    )

    # 6) Composite and get combined image
    combined = Image.alpha_composite(base, overlay)
    return combined


def build_radio_options(candidate_ids):
    """
    Given bm.book_infos[book_id]['id'] (a list of ints or None),
    return a list of strings of the form "CALL_NUMBER, ALT_TITLE".
    Skip any None or missing entries. Then append the MANUAL option.
    """
    opts = []
    for cid in candidate_ids:
        if cid is None:
            continue
        info = book_database.get(str(cid))
        if not info:
            continue
        callnum   = info.get("call_number", "")
        alt_title = info.get("alt_title", "")
        if callnum or alt_title:
            opts.append(f"{callnum}, {alt_title}")
    # Always append the manual flag as the last radio option
    opts.append(MANUAL)
    return opts

def load_text(idx):
    return str(bm.book_infos[idx]["id"])

# moving to the next entry
def next_entry(state):
    img_i, fail_mode, u_book_i, s_group_i, s_book_i = state

    if img_i >= len(img_keys):
        return (
            gr.update(),            # img_u
            gr.update(visible=False),  # radio_u
            gr.update(visible=False),  # manual_dd
            gr.update(),            # img_s
            gr.update(),            # text_s
            gr.update(visible=False),  # grp_unsure
            gr.update(visible=False),  # grp_skipped
            state
        )

    img_key = img_keys[img_i]
    unsure_ids = img_groups[img_key]["unsure"]
    skipped_ids = img_groups[img_key]["skipped"]


    # UNSURE mode but there are no unsure IDs at all, switch to SKIPPED ---
    if fail_mode == "u" and not unsure_ids:
        return next_entry([img_i, "s", 0, 0, 0])

    if fail_mode == "s" and not skipped_ids:
        return next_entry([img_i + 1, "u", 0, 0, 0])
    
    mode_ids = unsure_ids if fail_mode == "u" else skipped_ids

    if fail_mode == "u":
        if u_book_i >= len(unsure_ids):
            return next_entry([img_i, "s", 0, 0, 0])
        book_id = unsure_ids[u_book_i]
    else:  # fail_mode == "s"
        # find between group
        if s_group_i >= len(skipped_ids):
            # next image
            return next_entry([img_i + 1, "u", 0, 0, 0])
        current_sublist = skipped_ids[s_group_i]
        # sub list (between) go next
        if s_book_i >= len(current_sublist):
            return next_entry([img_i, "s", 0, s_group_i + 1, 0])
        book_id = current_sublist[s_book_i]

    img = load_image_rgb(book_id)
    text = load_text(book_id)

    next_img_i    = img_i
    next_mode     = fail_mode
    next_u_book_i = u_book_i
    next_s_group  = s_group_i
    next_s_book   = s_book_i

    if fail_mode == "u":
        next_u_book_i += 1
        if next_u_book_i >= len(unsure_ids):
            # we’ve shown all “unsure” for this image → go to first skipped sublist
            next_mode     = "s"
            next_u_book_i = 0
            next_s_group  = 0
            next_s_book   = 0
    else:  # “s”
        next_s_book += 1
        current_sublist = skipped_ids[s_group_i]
        if next_s_book >= len(current_sublist):
            # finished this sublist → advance to next sublist
            next_s_group += 1
            next_s_book  = 0
            # If that was the last sublist, next call to next_entry will push to next image

    new_state = [next_img_i, next_mode, next_u_book_i, next_s_group, next_s_book]

    if fail_mode == "u":          # UNSURE layout now active
       # point on book
        point = bm.book_positions[book_id]
        img = annotate_on_image(img, point)
  
        candidate_ids = bm.book_infos[book_id]["id"]
        radio_labels = build_radio_options(candidate_ids)
        radio_update = gr.update(choices=radio_labels, value=None, visible=True)

        # dropdown stays hidden until user picks MANUAL
        manual_update = gr.update(choices=build_manual_choices(),
                                  value=None,
                                  visible=False)
        return (
            # 1) img_u: show annotated image
            gr.update(value=img, visible=True),
            # 2) radio_u: show the matching‐candidates radio
            radio_update,
            # 3) manual_dd: keep hidden until user picks “MANUAL”
            manual_update,

            # 4) img_s: hide (we’re in UNSURE mode)
            gr.update(visible=False),
            # 5) text_s: hide
            gr.update(visible=False),

            # 6) found_radio: hide (only used in SKIPPED)
            gr.update(visible=False),
            # 7) status_skipped: hide / clear
            gr.update(value="", visible=False),

            # 8) grp_unsure: show
            gr.update(visible=True),
            # 9) grp_skipped: hide
            gr.update(visible=False),

            # 10) state: return the updated state
            new_state
        )

    else:                    # SKIPPED layout active
        # load the base PIL image
        base_img = load_image_rgb(book_id)  # returns RGB → convert in helper
        # overlay skip‐box
        img_with_box = annotate_skip_box(base_img, book_id)
        skipped_str = "skipped " + load_text(book_id)

        
        return (
            # 1) Hide img_u
            gr.update(visible=False),
            # 2) Hide radio_u
            gr.update(visible=False),
            # 3) Hide manual_dd
            gr.update(visible=False),

            # 4) Show the masked image in img_s
            gr.update(value=img_with_box, visible=True),
            # 5) Show the “skipped {text}” Markdown
            gr.update(value=skipped_str, visible=True),
            # 6) Show found_radio (“Is this book in the masked area?”)
            gr.update(visible=True),
            # 7) Clear & show status_skipped
            gr.update(value="", visible=True),

            # 8) Hide grp_unsure
            gr.update(visible=False),
            # 9) Show grp_skipped
            gr.update(visible=True),

            # 10) Return the updated state
            new_state
        )

def on_radio_change(choice):
    if choice == MANUAL:
        return gr.update(visible=True)
    return gr.update(visible=False)

def record_and_advance(radio_choice, manual_choice, state):
    """
    1) If state is “UNSURE” and nothing is picked → bail with a warning.
    2) Otherwise, record the chosen book_id into `new_bm`, then call next_entry().
    """
    img_i, fail_mode, book_i = state

    # If we’re already past the last image-group, just call next_entry() → no record
    if img_i >= len(img_keys):
        base_updates = next_entry(state)
        return (*base_updates, gr.update(value=""))

    # Only enforce “must pick something” if we’re in UNSURE mode:
    if fail_mode == "u":
        # Nothing selected
        if (radio_choice is None) or (radio_choice == MANUAL and (not manual_choice)):
            # Don’t advance; keep all visible values unchanged, but show warning:
            return (
                gr.update(),            # img_u: no change
                gr.update(),            # radio_u: no change
                gr.update(),            # manual_dd: no change
                gr.update(visible=False),  # img_s  (still hidden)
                gr.update(visible=False),  # text_s (still hidden)
                gr.update(visible=True),    # grp_unsure: still visible
                gr.update(visible=False),   # grp_skipped: still hidden
                state,                      # state unchanged
                gr.update(value="⚠️ Please select a match or choose Manual first")
            )

        # user did pick something valid:
        if radio_choice and (radio_choice != MANUAL):
            # parse out the call_number from the selected radio string:
            callnum = radio_choice.split(",")[0].strip()
            chosen_id = int(call_to_id.get(callnum))
        else:
            chosen_id = int(manual_choice)
        
        chosen_id = int(chosen_id)  # ensure it’s an integer

        # TODO: add book
        # position = bm.book_positions[]
        # info = 
        # img_info = 
        # new_bm.add_book()
        # book_infos.append({"id": [chosen_id]})
        # new_bm.book_img_info.append(bm.book_img_info[img_i])  

    base_updates = next_entry(state)
    return (*base_updates, gr.update(value=""))

# ------------------------------------------------------------------ gradio UI

# ─── 3) Build the Gradio UI, including radio.change(…) to show/hide manual_dd ───
with gr.Blocks(title="Book browser") as demo:
    gr.Markdown("### Browse entries")
    state = gr.State(init_state)

    with gr.Group(visible=True) as grp_unsure:
        img_u    = gr.Image(type="pil", height=300, label="Unsure")
        radio_u  = gr.Radio(
            choices=[],
            label="Choose one of these matches:",
            visible=False,
            interactive=True,
        )
        manual_dd = gr.Dropdown(
            choices=[],
            label="Manual entry:",
            value=None,
            visible=False,
            allow_custom_value=False,
            interactive=True,
        )

        status_u = gr.Markdown(value="", visible=True)   # ← NEW: for warnings


        # Whenever the radio value changes, run on_radio_change to toggle manual_dd
        radio_u.change(
            on_radio_change,
            inputs=[radio_u],
            outputs=[manual_dd],
            queue=False
        )

    with gr.Group(visible=False) as grp_skipped:
        # ── 1) Non-interactive “masked” image ────────────────────────────
        img_s = gr.Image(
            type="pil",
            height=300,
            label="Skipped",
            visible=False
        )

        # ── 2) Interactive version (same spot/size), initially hidden ───
        #      (remove the `tool="editor"` line)
        img_clickable = gr.Image(
            type="pil",
            height=300,
            label="Click where the book is",
            interactive=True,
            visible=False
        )

        text_s = gr.Markdown(visible=False)  # e.g. “skipped {book_id}”

        found_radio = gr.Radio(
            choices=["Yes", "No"],
            label="Is this book in the masked area?",
            visible=False,
            interactive=True
        )

        status_skipped = gr.Markdown(value="", visible=True)

        # ── CALLBACK: user answers Yes/No ────────────────────────────────
        def on_found_radio(answer, state):
            img_i, mode, ub, sg, sb = state
            book_id = img_groups[img_keys[img_i]]["skipped"][sg][sb]

            if answer == "No":
                base = next_entry(state)
                return (*base[:-1], base[-1])  # the new state at the end

            # “Yes” → show the clickable overlay, keep the same state
            base_img     = load_image_rgb(book_id)
            img_with_box = annotate_skip_box(base_img, book_id)
            skipped_str  = f"skipped {load_text(book_id)}"

            return (
                gr.update(visible=False),                    # img_s
                gr.update(visible=False),                    # found_radio
                gr.update(value=img_with_box, visible=True), # img_clickable
                gr.update(value="Please click on the book location.", visible=True),  # status_skipped
                gr.update(value=skipped_str, visible=True),  # text_s
                gr.update(visible=True),                     # grp_skipped
                state                                        # <-- raw list, not .update()
            )



        found_radio.change(
            on_found_radio,
            inputs=[found_radio, state],
            outputs=[
                img_s,           # show/hide static image
                found_radio,     # show/hide the radio
                img_clickable,   # show/hide interactive image
                status_skipped,  # warning prompt
                text_s,          # “skipped {book_id}” text
                grp_skipped,     # keep group visible
                state
            ],
            queue=False
        )

        # ── CALLBACK: user clicks on img_clickable ───────────────────────
        def on_click_book(evt, state):
            # 1) Unpack the state tuple
            img_i, fail_mode, u_book_i, s_group_i, s_book_i = state

            # 2) Find which shelf‐photo group we’re in
            img_key     = img_keys[img_i]
            skipped_ids = img_groups[img_key]["skipped"]  # a list of lists

            # 3) Grab the current sublist and the specific book index
            current_sublist = skipped_ids[s_group_i]
            book_id         = current_sublist[s_book_i]

            # 4) Now evt.index gives you (x, y)
            x, y = evt.index

            # 5) Save that click
            new_bm.add_book(book_id, x, y)

            # 6) Advance to the next entry
            base_updates = next_entry(state)
            # base_updates is a tuple of 8 component updates + the new state

            # 7) Return exactly the same number of outputs,
            #    clearing any status message as the last update.
            return (
                *base_updates[:-1],        # the first N–1 updates
                gr.update(value=""),       # clear status_skipped (last visible component)
                base_updates[-1]           # the updated state
            )

        img_clickable.select(
            on_click_book,          # callback(evt, state)
            inputs=[state],         # pass current state in
            outputs=[
                img_u, radio_u, manual_dd,
                img_s, text_s,
                grp_unsure, grp_skipped,
                state,              # ← now only this callback updates state
                status_skipped
            ],
            queue=False
        )

        # ── END of grp_skipped group ────────────────────────────────────

    nxt   = gr.Button("Next →")

    # First load on page render
    demo.load(
        next_entry,
        inputs=[state],
        outputs=[
            img_u,           # 1
            radio_u,         # 2
            manual_dd,       # 3

            img_s,           # 4
            text_s,          # 5

            found_radio,     # 6   <— new
            status_skipped,  # 7   <— new

            grp_unsure,      # 8
            grp_skipped,     # 9

            state            # 10
        ],
        queue=False
    )



    # Subsequent clicks
    nxt.click(
        next_entry,
        inputs=[state],
        outputs=[
            img_u,           # 1
            radio_u,         # 2
            manual_dd,       # 3

            img_s,           # 4
            text_s,          # 5

            found_radio,     # 6
            status_skipped,  # 7

            grp_unsure,      # 8
            grp_skipped,     # 9

            state            # 10
        ],
    )
demo.launch()


/Users/isabelsieh/Desktop/library/.conda/lib/python3.11/site-packages/gradio/utils.py:1018: UserWarning: Expected 2 arguments for function <function on_click_book at 0x179a767a0>, received 1.
  warnings.warn(
/Users/isabelsieh/Desktop/library/.conda/lib/python3.11/site-packages/gradio/utils.py:1022: UserWarning: Expected at least 2 arguments for function <function on_click_book at 0x179a767a0>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


In [175]:
# ------------------------------------------------------------
# 0. prep – original (read-only) bm and an empty bm to save to
# ------------------------------------------------------------
new_bm  = BookMemory()          # where we’ll store the user’s final labels

# helper once, near the top ----------------------------------
call_to_id = {row[0]: i for i, row in enumerate(book_database)}   # "DT 515…" → 0 …

TOTAL   = len(bm.book_infos)    # how many books in the pile
MANUAL  = "Manually label book"
SKIP    = "Skip →"               # button label

chosen = [None] * TOTAL          # None until user labels each book

shortcut_js = """
<script>
document.addEventListener("keydown", (e) => {
  /* If the user is typing in an editable field, ignore the keystroke */
  const tag = e.target.tagName;
  if (["INPUT","TEXTAREA","SELECT"].includes(tag)) return;

  /* Grab the root *at the moment of the key-press* (it now exists) */
  const app  = document.querySelector("gradio-app");
  if (!app) return;                               // still not ready
  const root = app.shadowRoot || document;        // 5.x uses shadow DOM

  /* 1–9  →  nth radio button */
  if (/^[1-9]$/.test(e.key)) {
    const idx    = Number(e.key) - 1;
    const radios = root.querySelectorAll("#book_choices input[type='radio']");
    if (idx < radios.length) radios[idx].click();
  }

  /* Enter or →  submits  |  ←  goes back */
  if (e.key === "Enter" || e.key === "ArrowRight")
      root.querySelector("#submit_btn")?.click();

  if (e.key === "ArrowLeft")
      root.querySelector("#back_btn")?.click();
});
</script>
"""


# ------------------------------------------------------------
# 1. helpers
# ------------------------------------------------------------
def review_msg():
    """Return the banner that appears on the review/finalize page."""
    skipped = chosen.count(None)
    if skipped:
        return f"⚠️ **{skipped} book{'s' if skipped > 1 else ''} not matched** – go back to label them."
    return "🎉 All books matched – review or press **Finalize**."

def build_manual_choices():
    """
    Return a list of (label, value) tuples for every book in book_database,
    where label = "CALL_NUMBER, ALT_TITLE" and value = the database ID.
    """
    opts = []
    for db_id, rec in book_database.items():
        callnum   = rec.get("call_number", "")
        alt_title = rec.get("alt_title", "")
        if callnum or alt_title:
            label = f"{callnum}, {alt_title}"
            opts.append((label, db_id))
    return opts

def _as_list(x):
    """
    Always return a list.

    • None     → []
    • list/tuple → shallow-copied list(x)
    • scalar   → [x]
    """
    if x is None:
        return []
    if isinstance(x, (list, tuple)):
        return list(x)
    return [x]

def record_str(rec: dict) -> str:
    """"<call_number>, <alt_title>"  (skip the 'lang' field)."""
    return f"{rec['call_number']}, {rec['alt_title']}"

# maps call-number ➜ integer key used in bm.book_infos
call_to_id = {v["call_number"]: int(k) for k, v in book_database.items()}

def build_choices(info):
    ids  = _as_list(info.get("id"))
    sims = _as_list(info.get("similarity"))

    choices = []
    for bid, sim in zip(ids, sims):
        if bid is None:
            continue
        rec  = book_database[str(bid)]
        disp = f"{rec['call_number']}, {rec['alt_title']}"
        if sim is not None:
            disp += f"\n{sim*100:.1f}% similarity"
        choices.append((disp, disp))
    return choices

def ui_payload(idx):
    if idx < 0 or idx >= TOTAL:
        return None, gr.update(visible=False), gr.update(visible=False), ""

    info    = bm.book_infos[idx]
    img     = cv2.cvtColor(bm.book_img_info[idx]["image"], cv2.COLOR_BGR2RGB)
    sel     = chosen[idx]            # previously chosen label (or None)
    choices = build_choices(info)

    # ——————————————————————————————————  a) we have candidate matches  ——
    if choices:                    # (a) there are radio suggestions
        choices.append(MANUAL)

        radio_val  = None          # ← default = None, not ""
        manual_val = None
        manual_vis = False

        if sel:
            if sel in [c[1] for c in choices]:
                radio_val = sel
            else:                  # came from manual path earlier
                radio_val  = MANUAL
                manual_val = sel
                manual_vis = True

        return (
            img,
            gr.update(choices=choices, value=radio_val, visible=True),
            gr.update(value=manual_val,  visible=manual_vis),
            f"*Book {idx+1} of {TOTAL}*",
        )

    # (b) no matches
    status_msg = (
        f"*Book {idx+1} of {TOTAL}*\n\n"
        "**No possible matches found – please type the book info below.**"
    )
    manual_val = sel or None

    return (
        img,
        # radio has exactly one hidden option, auto-selected
        gr.update(
            choices=[MANUAL],
            value=MANUAL,
            visible=False          # ← user never sees the list
        ),
        gr.update(
            value=manual_val,
            visible=True           # dropdown shown
        ),
        status_msg,
    )


# ------------------------------------------------------------
# 2. callbacks
# ------------------------------------------------------------
def submit_and_next(radio_choice, manual_text, idx):
    manual_text = (manual_text or "").strip()

    if (radio_choice is None or radio_choice == MANUAL) and not manual_text:
        warn = "⚠️ Please label book"

        return (
            gr.update(),          # image – no change
            gr.update(),          # radio  – no change
            gr.update(),          # dropdown – no change
            warn,                 # status markdown
            idx,                  # current_idx unchanged
            gr.update(visible=False),   # Finalize stays hidden
            gr.update(visible=True),    # Go Back still visible
            gr.update(visible=True),    # Match Book still visible
        )

    label        = manual_text if manual_text else radio_choice   # full label
    call_number  = label.split(",")[0].strip()                    # first token
    book_id      = call_to_id.get(call_number)                    # lookup row #

    chosen[idx] = label                                           # for UI

    # ── keep new_bm in sync: store both id *and* call ──────────
    if idx < len(new_bm.book_infos):               # overwrite existing slot
        new_bm.book_infos[idx]["id"]   = [book_id]
        new_bm.book_infos[idx]["call"] = call_number
    else:                                          # first time for this slot
        new_bm.add_book(
            np.array([0, 0, 0]),
            {
                "id":   [book_id],
                "call": call_number,
                "similarity": [],
                "confidence": [],
            },
            bm.book_img_info[idx],
        )

    # advance index
    idx += 1
    if idx >= TOTAL:                                # review / finalize page
        idx = TOTAL                                 # mark “review” state
        return (
            gr.update(visible=False),               # img hidden
            gr.update(visible=False),               # radio hidden
            gr.update(visible=False),               # dropdown hidden
            review_msg(),
            idx,
            gr.update(visible=True),                # Finalize btn shows
            gr.update(visible=True),                # ← Go Back stays visible
            gr.update(visible=False),               # Match Book HIDDEN  ← NEW
        )

    # ------------------------------------------------
    #  ❖  Still more spines to label
    # ------------------------------------------------
    img, r_upd, m_upd, stat = ui_payload(idx)
    return (
        img,            # image
        r_upd,          # radio
        m_upd,          # dropdown
        stat,           # status markdown
        idx,            # current_idx
        gr.update(visible=False),   # Finalize hidden until all done
        gr.update(visible=True),    # Go Back visible
        gr.update(visible=True),    # Match Book visible
    )

def skip_and_next(idx):
    """
    Advance without storing a label.
    The frame it returns is identical to submit_and_next,
    except that chosen[idx] stays None.
    """
    idx += 1
    if idx >= TOTAL:                      # finished – go to review page
        idx = TOTAL
        return (
            gr.update(visible=False),     # img
            gr.update(visible=False),     # radio
            gr.update(visible=False),     # dropdown
            review_msg(),                 # status markdown
            idx,
            gr.update(visible=True),      # Finalize button
            gr.update(visible=True),      # Go Back
            gr.update(visible=False),     # Match Book (hidden)
        )

    # still books left → reuse your existing payload builder
    img, r_upd, m_upd, stat = ui_payload(idx)
    return (
        img, r_upd, m_upd, stat, idx,
        gr.update(visible=False),         # Finalize hidden
        gr.update(visible=True),          # Go Back visible
        gr.update(visible=True),          # Match Book visible
    )


def radio_updated(choice):
    if choice == MANUAL:
        return gr.update(visible=True)        # keep whatever is in the box
    return gr.update(value=None, visible=False)   # clear with None


def go_back(idx):
    if idx == TOTAL:                # from review page → last spine
        idx = TOTAL - 1
    else:
        idx = max(idx - 1, 0)

    img, r_upd, m_upd, stat = ui_payload(idx)
    return (
        gr.update(value=img, visible=True),   # image
        r_upd,                                # radio
        m_upd,                                # dropdown
        stat,                                 # status markdown
        idx,                                  # current_idx
        gr.update(visible=False),             # Finalize hidden (unless idx==TOTAL)
        gr.update(visible=True),              # Go Back visible
        gr.update(visible=True),              # Match Book visible
    )


# ------------------------------------------------------------
# after you finish laying out the interface …
# ------------------------------------------------------------

def on_start():
    img, r_upd, m_upd, stat = ui_payload(0)

    return (
        img,                      # 1. image
        r_upd,                    # 2. radio update
        m_upd,                    # 3. dropdown update
        stat,                     # 4. status markdown
        0,                        # 5. current_idx (State)
        gr.update(visible=False), # 6. Finalize button (hidden at start)
        gr.update(visible=True),  # 7. Go Back button  (shown)
        gr.update(visible=True),  # 8. Match Book button (shown)
    )

import json, pprint   # at top of file if not already imported

def finalize_fn():
    if chosen.count(None):
        return (
            gr.update(), gr.update(), gr.update(), gr.update(), gr.update(),
            "⚠️ Cannot finalize – there are still unlabeled books.",
        )
    # ── DEBUG  ──────────────────────────────────────────────
    print("\n===== new_bm contents =====")
    pprint.pprint(new_bm.book_infos, compact=True, width=120)
    # or, if you prefer JSON:
    # print(json.dumps(new_bm.book_infos, indent=2))
    print("================================\n")
    # ─────────────────────────────────────────────────────────

    lock = gr.update(interactive=False)
    return (
        lock,        # radio
        lock,        # dropdown
        lock,        # Match Book
        lock,        # Go Back
        lock,        # Finalize
        "✅ Matches locked. Thank you!",
    )


# ------------------------------------------------------------
# 3. interface
# ------------------------------------------------------------
CSS = """
#book_choices label span { white-space: pre-wrap; }
body ul.options[role="listbox"] { max-height:220px!important;overflow-y:auto!important;overflow-x:hidden!important;}
"""

with gr.Blocks(title="Book Matcher", css=CSS, head=shortcut_js) as demo:
    gr.Markdown("## Book Matcher")

    current_idx = gr.State(0)

    with gr.Row():
        with gr.Column(scale=3):
            img_out = gr.Image(label="Book Image", height=320)

        with gr.Column(scale=2):
            status = gr.Markdown()

            # add elem_id here ⬇
            radio  = gr.Radio(
                label="Possible Labels",
                elem_id="book_choices"
            )

            manual = gr.Dropdown(
                label="Type or pick title",
                choices=[record_str(v) for v in book_database.values()],   # ← only this line changed
                filterable=True,
                allow_custom_value=False,
                interactive=True,
                visible=False,
                elem_id="manual_dd",
            )

            with gr.Row():
                back   = gr.Button("← Go Back", elem_id="back_btn")
                submit = gr.Button("Match Book →", elem_id="submit_btn")
                skipbn = gr.Button("Skip →",        elem_id="skip_btn")   # ← NEW
                finalize = gr.Button(
                    "Finalize matches (locked)",
                    elem_id="finalize_btn",
                    visible=False,
                )

    # wire callbacks
    submit.click(
        submit_and_next,
        inputs=[radio, manual, current_idx],
        outputs=[img_out, radio, manual, status,
                current_idx, finalize, back, submit],   # note order
    )

    back.click(
        go_back,
        inputs=current_idx,
        outputs=[
            img_out,   # updated image (value + visible flag)
            radio,     # radio update
            manual,    # dropdown update
            status,    # status markdown
            current_idx,
            finalize,  # Finalize button (its visible flag changes here)
            back,      # Go Back button (visibility may change)
            submit,    # Match Book button (visibility may change)
        ],
    )

    skipbn.click(
    skip_and_next,
    inputs=current_idx,
    outputs=[
        img_out, radio, manual, status,
        current_idx, finalize, back, submit,
    ],
    )

    finalize.click(
        finalize_fn,
        inputs=None,
        outputs=[radio, manual, submit, back, finalize, status],
    )

    radio.change(
    radio_updated,   # function to call
    inputs=radio,    # value coming from the radio component
    outputs=manual,  # update only the dropdown
    )

    demo.load(
        on_start,
        inputs=None,
        outputs=[
            img_out,
            radio,
            manual,
            status,
            current_idx,
            finalize,  # hidden at startup
            back,      # visible
            submit,    # visible
        ],
    )

# (nothing else needed down here)
demo.launch(share=True)                # start the Gradio app


* Running on local URL:  http://127.0.0.1:7951
* Running on public URL: https://9678417d125baca907.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
